In [7]:
import numpy as np
np.set_printoptions(precision = 3, linewidth = 150)

In [25]:
np.random.seed(4)
counts = np.repeat(
    np.arange(5, 10),
    [8, 4, 4, 3, 1])
data = np.random.poisson(counts[None, :], [100, 20]).astype(np.float32)
data -= np.mean(data, axis=0)
cov = np.cov(data.T).astype(np.float32)
print('{} {}'.format(cov.shape, cov.dtype))

(20, 20) float32


In [26]:
A = np.diag(np.linalg.eigh(cov[:8, :8])[0])
A_aug = np.c_[np.zeros([A.shape[0]+1,1]), np.r_[np.zeros_like(A[0:1, :]), A]]
A_aug

array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 2.71 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 3.104, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 3.865, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 4.243, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 4.733, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 5.348, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 5.814, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 6.598]])

In [27]:
from l0pca.bunch import rank_one_update

In [28]:
update_aug = rank_one_update.create_update_append_column(
    np.arange(8), np.sqrt(np.diagonal(A)),
    np.linalg.eigh(cov[:8, :8])[1], 8, cov,
    np.linalg.norm(data, axis=0), 100).numpy()
update_aug

array([ 2.23 , -0.277,  0.369,  0.099, -0.182,  0.116, -0.006,  0.37 , -0.273], dtype=float32)

In [49]:
np.linalg.eigh(cov[:8, :8])[0]

array([2.71 , 3.104, 3.865, 4.243, 4.733, 5.348, 5.814, 6.598], dtype=float32)

In [29]:
np.linalg.eigh(cov[:9, :9])[0]

array([2.578, 2.951, 3.837, 4.13 , 4.638, 4.983, 5.348, 6.242, 7.158], dtype=float32)

In [30]:
np.linalg.eigh(A_aug + update_aug[:, None].dot(update_aug[None, :]))[0]

array([2.578, 2.951, 3.837, 4.13 , 4.638, 4.983, 5.348, 6.242, 7.158])

In [44]:
np.linalg.eigh(A_aug + np.linalg.norm(update_aug) ** 1 * np.c_[update_aug[:, None], np.zeros_like(A_aug[:, 1:])])[0]

array([2.39 , 2.909, 3.833, 4.128, 4.652, 5.044, 5.348, 6.251, 7.066])

In [48]:
np.square(np.linalg.svd(np.sqrt(A_aug))[1][::-1])

array([0.   , 2.71 , 3.104, 3.865, 4.243, 4.733, 5.348, 5.814, 6.598])

In [52]:
np.square(
    np.linalg.svd(np.sqrt(A_aug) + np.linalg.norm(update_aug) ** 0 * np.c_[update_aug[:, None], np.zeros_like(A_aug[:, 1:])])[1]
)[::-1]

array([2.578, 2.951, 3.837, 4.13 , 4.638, 4.983, 5.348, 6.242, 7.158])

In [53]:
np.sqrt(A_aug) + np.linalg.norm(update_aug) ** 0 * np.c_[update_aug[:, None], np.zeros_like(A_aug[:, 1:])]

array([[ 2.23 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.277,  1.646,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.369,  0.   ,  1.762,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.099,  0.   ,  0.   ,  1.966,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.182,  0.   ,  0.   ,  0.   ,  2.06 ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.116,  0.   ,  0.   ,  0.   ,  0.   ,  2.176,  0.   ,  0.   ,  0.   ],
       [-0.006,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  2.313,  0.   ,  0.   ],
       [ 0.37 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  2.411,  0.   ],
       [-0.273,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  2.569]])

In [76]:
A_aug_inv = np.linalg.inv(np.sqrt(A_aug) + np.linalg.norm(update_aug) ** 0 * np.c_[update_aug[:, None], np.zeros_like(A_aug[:, 1:])])
A_aug_inv

array([[ 0.448,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.075,  0.607,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.094,  0.   ,  0.568,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.023,  0.   ,  0.   ,  0.509,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.04 ,  0.   ,  0.   ,  0.   ,  0.485,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.024,  0.   ,  0.   ,  0.   ,  0.   ,  0.46 ,  0.   ,  0.   ,  0.   ],
       [ 0.001,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.432,  0.   ,  0.   ],
       [-0.069,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.415,  0.   ],
       [ 0.048,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.389]])

In [57]:
np.linalg.inv(np.sqrt(A_aug) - np.diag([1,1,1,1,1,1,1,1,1]) + np.linalg.norm(update_aug) ** 0 * np.c_[update_aug[:, None], np.zeros_like(A_aug[:, 1:])])

array([[ 0.813,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.348,  1.547,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.393,  0.   ,  1.313,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.083,  0.   ,  0.   ,  1.035,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.14 ,  0.   ,  0.   ,  0.   ,  0.943,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.08 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.851,  0.   ,  0.   ,  0.   ],
       [ 0.004,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.762,  0.   ,  0.   ],
       [-0.213,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.709,  0.   ],
       [ 0.142,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.638]])

In [71]:
list(map(float, np.sqrt(np.linalg.eigh(cov[:9, :9])[0])))

[1.6055138111114502,
 1.7179895639419556,
 1.958818793296814,
 2.0323104858398438,
 2.153677463531494,
 2.2322957515716553,
 2.3126745223999023,
 2.498466968536377,
 2.6754770278930664]

In [65]:
lam = 2.6754770278930664
np.linalg.svd(
    np.linalg.inv(np.sqrt(A_aug) - np.diag([lam,lam,lam,lam,lam,lam,lam,lam,lam]) + np.linalg.norm(update_aug) ** 0 * np.c_[update_aug[:, None], np.zeros_like(A_aug[:, 1:])]))[1]

array([11.221,  4.756,  2.756,  2.107,  1.833,  1.509,  1.381,  1.025,  0.878])

In [73]:
lam = 2.498466968536377
np.linalg.svd(
    np.linalg.inv(np.sqrt(A_aug) - np.diag([lam,lam,lam,lam,lam,lam,lam,lam,lam]) + np.linalg.norm(update_aug) ** 0 * np.c_[update_aug[:, None], np.zeros_like(A_aug[:, 1:])]))[1]

array([25.366, 12.962,  5.382,  3.255,  2.6  ,  2.014,  1.809,  1.247,  1.016])

In [75]:
list(map(float, np.linalg.eigh(cov[:9, :9])[0]))

[2.577674627304077,
 2.9514880180358887,
 3.8369710445404053,
 4.13028621673584,
 4.638326168060303,
 4.983144283294678,
 5.348463535308838,
 6.242337226867676,
 7.158176898956299]

In [85]:
A_aug_full = np.sqrt(A_aug) + np.c_[update_aug[:, None], np.zeros_like(A_aug[:, 1:])]
A_aug_full

array([[ 2.23 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.277,  1.646,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.369,  0.   ,  1.762,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.099,  0.   ,  0.   ,  1.966,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.182,  0.   ,  0.   ,  0.   ,  2.06 ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.116,  0.   ,  0.   ,  0.   ,  0.   ,  2.176,  0.   ,  0.   ,  0.   ],
       [-0.006,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  2.313,  0.   ,  0.   ],
       [ 0.37 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  2.411,  0.   ],
       [-0.273,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  2.569]])

In [92]:
np.linalg.inv(A_aug_full.dot(A_aug_full.T) - np.diag(np.repeat(7.158, 9)))

array([[ 1.276e+03, -2.549e+02,  3.726e+02,  1.233e+02, -2.558e+02,  1.960e+02, -1.347e+01,  1.127e+03, -1.998e+03],
       [-2.549e+02,  5.067e+01, -7.440e+01, -2.462e+01,  5.108e+01, -3.913e+01,  2.689e+00, -2.251e+02,  3.989e+02],
       [ 3.726e+02, -7.440e+01,  1.085e+02,  3.599e+01, -7.467e+01,  5.719e+01, -3.931e+00,  3.291e+02, -5.832e+02],
       [ 1.233e+02, -2.462e+01,  3.599e+01,  1.161e+01, -2.471e+01,  1.893e+01, -1.301e+00,  1.089e+02, -1.930e+02],
       [-2.558e+02,  5.108e+01, -7.467e+01, -2.471e+01,  5.093e+01, -3.927e+01,  2.699e+00, -2.260e+02,  4.004e+02],
       [ 1.960e+02, -3.913e+01,  5.719e+01,  1.893e+01, -3.927e+01,  2.967e+01, -2.067e+00,  1.731e+02, -3.067e+02],
       [-1.347e+01,  2.689e+00, -3.931e+00, -1.301e+00,  2.699e+00, -2.067e+00, -4.105e-01, -1.189e+01,  2.108e+01],
       [ 1.127e+03, -2.251e+02,  3.291e+02,  1.089e+02, -2.260e+02,  1.731e+02, -1.189e+01,  9.950e+02, -1.765e+03],
       [-1.998e+03,  3.989e+02, -5.832e+02, -1.930e+02,  4.004e+

In [98]:
np.linalg.eigh(np.linalg.inv(A_aug_full.dot(A_aug_full.T) - np.diag(np.repeat(6.242337226867676, 9))))[0]

array([-1.119e+00, -7.942e-01, -6.234e-01, -4.735e-01, -4.157e-01, -3.039e-01, -2.729e-01,  1.092e+00,  7.126e+06])

In [101]:
np.linalg.eigh(cov[:9, :9])[1][:, -2]

array([ 0.062,  0.211, -0.378,  0.228,  0.403, -0.343,  0.334, -0.508, -0.327], dtype=float32)

In [99]:
np.linalg.eigh(np.linalg.inv(A_aug_full.dot(A_aug_full.T) - np.diag(np.repeat(6.242337226867676, 9))))[1][:, -1]

array([ 0.292, -0.064,  0.096,  0.034, -0.074,  0.063, -0.005,  0.705,  0.628])

# sparse_mat * sparse_mat.T * x - mu .* x = guess

sparse_mat.T * x - (inv(sparse_mat) .* mu) * x = inv(sparse_mat) * guess

In [102]:
np.linalg.inv(A_aug_full)

array([[ 0.448,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.075,  0.607,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.094,  0.   ,  0.568,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.023,  0.   ,  0.   ,  0.509,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.04 ,  0.   ,  0.   ,  0.   ,  0.485,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.024,  0.   ,  0.   ,  0.   ,  0.   ,  0.46 ,  0.   ,  0.   ,  0.   ],
       [ 0.001,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.432,  0.   ,  0.   ],
       [-0.069,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.415,  0.   ],
       [ 0.048,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.389]])

In [104]:
guess = np.random.normal([9, 1])
guess /= np.linalg.norm(guess)
mu = 6.242337226867676

In [110]:
# Upper diagonalization (subtracting X times last row) 
A_aug_full.T - np.linalg.inv(A_aug_full) * mu

array([[-0.57 , -0.277,  0.369,  0.099, -0.182,  0.116, -0.006,  0.37 , -0.273],
       [-0.47 , -2.145,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.586,  0.   , -1.781,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.141,  0.   ,  0.   , -1.21 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.247,  0.   ,  0.   ,  0.   , -0.97 ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.149,  0.   ,  0.   ,  0.   ,  0.   , -0.694,  0.   ,  0.   ,  0.   ],
       [-0.007,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.387,  0.   ,  0.   ],
       [ 0.429,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.178,  0.   ],
       [-0.298,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.138]])